# Tensorflow Dataset API를 활용하여 Kaggle 데이터 학습 시켜보기
## 예제 : [Invasive Species Monitoring](https://www.kaggle.com/c/invasive-species-monitoring)

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# pillow
from PIL import Image
# opencv-python
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cd 00_project

E:\dl_study\00_project


### 1. pandas 라이브러리를 통해 라벨 정보가 들어있는 csv 파일을 읽어서 file_list에 저장

In [3]:
train_dir = "./Invasive_Species/dataset/"

# pandas로 csv 파일 읽어오기
file_list = pd.read_csv(train_dir + 'train_labels.csv')
# pandas DataFrame의 data type 확인
print(type(file_list))
# csv 파일을 잘 읽어왔나 상위 10개의 데이터 확인
file_list.head(10)

<class 'pandas.core.frame.DataFrame'>


,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1
5,6,0
6,7,1
7,8,1
8,9,0
9,10,0


pandas로 파일 이름과 라벨 정보를 읽어오는 테스트로  
5번째 파일의 이름과 ----> file_list.iloc[4][0]  
5번째 파일의 라벨을 ----> file_list.iloc[4][1]  
시험삼아 출력


In [4]:
# DataFrame.head() 로 출력한 결과와 일치하는가 확인
print(file_list.iloc[4][0]) # [5번째 줄][name] 출력
print(file_list.iloc[4][1]) # [5번째 줄][invasive] 출력

5
1


### 2. Dataset 구성 밑준비
x_images에는 jpg 파일의 이름과 경로를 저장 (dtype=string)  
y_labels에는 class(0 또는 1)를 저장

In [5]:
# 뽑아올 샘플 수 설정 (임의로 100개만 사용)
n_samples = 100
# 빈 list형 변수 만들어두기
x_images, y_labels = [], []
# 샘플 수 만큼 for문 반복
for i in range(n_samples):
    # 파일이 저장된 경로와 이름, 확장자를 문자형(string)으로 변수 x_images에 차곡차곡 저장(append)
    x_images.append(train_dir + 'train/' + str(file_list.iloc[i][0]) + '.jpg')
    # y_labels에 invasive 정보 저장
    if file_list.iloc[i][1] == 0:
        # invasive가 0이면 y_labels에 0으로 저장
        y_labels.append(0)
    else:
        # invasive가 0이 아니면 y_labels에 1로 저장
        y_labels.append(1)

In [6]:
#잘 되었나 10개만 시험삼아 출력
x_images[:10]

['./Invasive_Species/dataset/train/1.jpg',
 './Invasive_Species/dataset/train/2.jpg',
 './Invasive_Species/dataset/train/3.jpg',
 './Invasive_Species/dataset/train/4.jpg',
 './Invasive_Species/dataset/train/5.jpg',
 './Invasive_Species/dataset/train/6.jpg',
 './Invasive_Species/dataset/train/7.jpg',
 './Invasive_Species/dataset/train/8.jpg',
 './Invasive_Species/dataset/train/9.jpg',
 './Invasive_Species/dataset/train/10.jpg']

In [7]:
y_labels[:10]

[0, 0, 1, 0, 1, 0, 1, 1, 0, 0]

### 3. Dataset 구성  
2에서 만든 파일 목록과 라벨 목록을 tensor로 만들기

In [8]:
tf_images = tf.constant(x_images)
tf_labels = tf.constant(y_labels)

Tensor로 만든 파일 목록과 라벨 목록으로 Dataset을 구성

In [9]:
train_data = tf.data.Dataset.from_tensor_slices((tf_images, tf_labels))

구성된 Dataset을 통해  
- 파일 목록에서 파일을 읽어오고 (tf.read_file)  
- 읽어온 이미지 파일을 decode (tf.image.decode_jpeg)  
- decode된 이미지를 resize (tf.image.resize_images)  
- 라벨 목록은 one hot encoding (tf.one_hot)  

In [10]:
def decode_images(image, label):
    x = tf.read_file(image) # file read 
    x = tf.image.decode_jpeg(x, channels=3) # 컬러 이미지라면 channels=3, 흑백이라면 channels=1
    x = tf.image.resize_images(x, [100, 100]) # 이미지 크기를 100 x 100으로 resize

    y = tf.one_hot(label, depth=2) # 라벨을 one hot vector로 만들어준다. class는 2개
    return x, y

### 4. Dataset 설정
- 학습에 사용될 Dataset을 위에서 구성한 decode_image function을 넣어주고(map)
- mini batch를 구성해준다. (batch)

In [11]:
train_data = train_data.map(decode_images).batch(10)

Dataset을 iterable하게 해주기 위해 Iterator를 만들고  
Iterator가 실행될 때마다 반환되는 값을 설정(get_next)  
iterator initializer에 사용할 dataset 설정

In [12]:
# iterator 만들기
data_iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
x_image, y_label = data_iterator.get_next()

# iterator 초기화를 위해 initializer 생성
init_iter = data_iterator.make_initializer(train_data)

In [13]:
# iterator에 type과 shape 확인
print(train_data.output_types)
print(train_data.output_shapes)
# tensorflow의 자료형은 아래와 같은 형태가 일반적임
# 이미지의 경우 : [batch][height][width][channel]
# 라벨의 경우  : [batch][class]
# 위와 같이 잘 나오나 확인

# batch은 기본적으로 None으로 보임

(tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(100), Dimension(100), Dimension(3)]), TensorShape([Dimension(None), Dimension(2)]))


### 5. 모델 구성

In [14]:
is_training = False

# build model to predict Y
conv = tf.layers.conv2d(x_image, 32, 3, 1, 'same', activation=tf.nn.relu)
conv = tf.layers.conv2d(x_image, 32, 3, 1, 'same', activation=tf.nn.relu)
conv = tf.layers.dropout(conv, 0.25, training=is_training)
conv = tf.layers.max_pooling2d(conv, 5, 5)
conv = tf.layers.conv2d(conv, 64, 3, 1, 'same', activation=tf.nn.relu)
conv = tf.layers.conv2d(conv, 64, 3, 1, 'same', activation=tf.nn.relu)
conv = tf.layers.dropout(conv, 0.25, training=is_training)
conv = tf.layers.max_pooling2d(conv, 5, 5)
conv = tf.layers.conv2d(conv, 128, 3, 1, 'same', activation=tf.nn.relu)
conv = tf.layers.conv2d(conv, 128, 3, 1, 'same', activation=tf.nn.relu)
conv = tf.layers.dropout(conv, 0.25, training=is_training)
conv = tf.layers.average_pooling2d(conv, 4, 1)
conv = tf.layers.flatten(conv)
logits = tf.layers.dense(conv, 2, name='Logits')

### 6. Define loss function / optimizer / prediction

In [15]:
with tf.name_scope('Loss'):
    entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_label, logits=logits)
    loss = tf.reduce_mean(entropy)

optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

with tf.name_scope('Prediction'):
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(y_label, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

### 7. 학습 시작 및 결과 출력

In [17]:
total_epochs = 10
step = 0

# session 시작
with tf.Session() as sess:  
    # 모든 변수(variables)를 초기화
    sess.run(tf.global_variables_initializer())

    # Tensorboard에 그래프 저장
    writer = tf.summary.FileWriter('./graphs', sess.graph)

    for i in range(total_epochs): 
        # 학습을 위한 Session 시작
        sess.run(init_iter) # Dataset API로 data를 넣어주기 위해 train data용 iterator 초기화 실행
        is_training = True
        total_loss = 0

        try:
            while True:
                _, l = sess.run([optimizer, loss])
                step += 1
                total_loss += l

        # train_data를 모두 사용해서 계산했다면 (MNIST의 train_data의 경우 55,000번)
        # tf.errors.OutOfRangeError가 발생하는데
        # 이 에러가 발생했다면 1번의 epoch이 끝났다는 것을 의미하기 때문에 아무 것도 안하고 그냥 넘어감(pass)
        except tf.errors.OutOfRangeError: 
            pass
        
        print('Epoch {0}: loss = {1:0.4f}'.format(i+1, total_loss/n_samples))
        
        # Test Session
        is_training = False
        sess.run(init_iter)
        total_correct_preds = 0
        try:
            while True:
                accuracy_batch = sess.run([accuracy])
                total_correct_preds += accuracy_batch
        except tf.errors.OutOfRangeError:
            pass

        print('Accuracy {0:0.4f}'.format(total_correct_preds/n_samples))

    writer.close()

Epoch 1: loss = 95.5413
Accuracy 0.6600
Epoch 2: loss = 0.0664
Accuracy 0.6600
Epoch 3: loss = 0.0654
Accuracy 0.6600
Epoch 4: loss = 0.0670
Accuracy 0.6600
Epoch 5: loss = 0.0622
Accuracy 0.6600
Epoch 6: loss = 0.0604
Accuracy 0.6600
Epoch 7: loss = 0.0571
Accuracy 0.6700
Epoch 8: loss = 0.0519
Accuracy 0.7900
Epoch 9: loss = 0.0448
Accuracy 0.8800
Epoch 10: loss = 0.0370
Accuracy 0.8300


input data가 적어서 accuracy는 충분치 않지만 loss가 줄어드는 것을 보아 잘 학습되고 있는 것으로 생각할 수 있음